In [3]:
%load_ext autoreload
%autoreload 2
import matplotlib
from matplotlib import rcParams, rc
import matplotlib.pyplot as plt
from matplotlib import colors
import pandas as pd
import numpy as np
from astropy.table import Table, join
from astropy.io import fits
from tqdm import tqdm

import cmasher as cmr
cmap = cmr.dusk

from tqdm import tqdm
import joblib 

rcParams["savefig.dpi"] = 150
rcParams["figure.dpi"] = 100
rcParams["font.size"] = 8
rcParams['font.family'] = 'sans-serif'
%config InlineBackend.figure_format = "retina"

import sys
sys.path.append("/home/jdli/transpec")

import time
import os
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import KFold
from kvxp.xpformer import XPformer, XPformer2
from kvxp.data import  GXP_AP_4lb, GXP_5lb, GXP_4lb
from kvxp.utils import *
from kvxp.vis import draw_hist2d

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
df = Table.read("/data/jdli/gaia/HWR_for_Jiandong.fits").to_pandas()

print(df.shape, df.columns)

(122962, 8) Index(['source_id', 'ra', 'dec', 'parallax', 'phot_g_mean_mag', 'mh_xgboost',
       'teff_xgboost', 'logg_xgboost'],
      dtype='object')


In [25]:
def recover_coef(series):
    return [list(map(float, s[1:-1].split(','))) for s in series]


def l2norm(y):
    return np.sqrt(np.sum(y**2, axis=1))

bp_scaler_name = "/home/jdli/transpec/models/scaler_bp_gmagand_0228.gz"
rp_scaler_name = "/home/jdli/transpec/models/scaler_rp_gmagand_0228.gz"
label_scaler_name = "/home/jdli/transpec/models/scaler_labels_0228.gz'"

coef_names = ['bp_coefficients', 'bp_coefficient_errors', 
              'rp_coefficients', 'rp_coefficient_errors']

pred_dfs = pd.DataFrame()
    
xp_coeff_array = np.zeros([4, len(df), 55])

xp_chunks = pd.read_csv(
    "/nfsdata/share/gaiaxp/gdr3_jdli_sid_xp_continuous_mean_spectrum.csv", 
    chunksize=int(1e6), sep="|")

df = Table.read("/home/jdli/transpec/data/GaiaDR3_SID_G16.fits").to_pandas()

save_dir = "/nfsdata/share/gaiaxp/"
save_name = "xp_2tian.npy"


for i,xp_chunk in tqdm(enumerate(xp_chunks)):
    
    pred_df  = pd.merge(df, xp_chunk, left_on='source_id', right_on='source_id')
    pred_dfs = pd.concat((pred_dfs, pred_df), axis='index')
    print(len(pred_dfs))


for col in coef_names:
    pred_dfs[col] = recover_coef(pred_dfs[col])
    
print(xp_coeff_array.shape)

1it [00:31, 31.16s/it]

0


2it [01:02, 31.08s/it]

0


3it [01:35, 31.90s/it]

...

0


16it [08:29, 31.82s/it]

0


17it [08:57, 31.64s/it]

0
(4, 122962, 55)


In [ ]:
xp_chunks = pd.read_csv(
    "/nfsdata/share/gaiaxp/gdr3_jdli_sid_xp_continuous_mean_spectrum.csv", 
    sep="|"
)